# Capstone Project: Comment Subtopics Analysis for Airbnb Hosts
---

How can a host on Airbnb understand that are their strengths and weaknesses? How can hosts point out the demand trend of their customers from a large scale of comments? This project focuses on using machine learning tools to help hosts understand the underlying trends of the comments on their property.  

---

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Capstone-Project:-Comment-Subtopics-Analysis-for-Airbnb-Hosts" data-toc-modified-id="Capstone-Project:-Comment-Subtopics-Analysis-for-Airbnb-Hosts-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Capstone Project: Comment Subtopics Analysis for Airbnb Hosts</a></span></li><li><span><a href="#Part-3.2:-LDA-Analysis-On-Comments-Based-On-Overall-Review" data-toc-modified-id="Part-3.2:-LDA-Analysis-On-Comments-Based-On-Overall-Review-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Part 3.2: LDA Analysis On Comments Based On Overall Review</a></span><ul class="toc-item"><li><span><a href="#Import-Review-Data" data-toc-modified-id="Import-Review-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Import Review Data</a></span><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Import-Review-Data-Steps" data-toc-modified-id="Import-Review-Data-Steps-2.1.0.1"><span class="toc-item-num">2.1.0.1&nbsp;&nbsp;</span>Import Review Data Steps</a></span></li></ul></li></ul></li><li><span><a href="#LDA-Prep" data-toc-modified-id="LDA-Prep-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>LDA Prep</a></span><ul class="toc-item"><li><span><a href="#Step1:-CountVectorize" data-toc-modified-id="Step1:-CountVectorize-2.2.1"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Step1: CountVectorize</a></span><ul class="toc-item"><li><span><a href="#Good-Rating-Dictionary" data-toc-modified-id="Good-Rating-Dictionary-2.2.1.1"><span class="toc-item-num">2.2.1.1&nbsp;&nbsp;</span>Good Rating Dictionary</a></span></li></ul></li><li><span><a href="#LDA-On-Good-Rating" data-toc-modified-id="LDA-On-Good-Rating-2.2.2"><span class="toc-item-num">2.2.2&nbsp;&nbsp;</span>LDA On Good Rating</a></span></li><li><span><a href="#Bad-Rating-Dictionary" data-toc-modified-id="Bad-Rating-Dictionary-2.2.3"><span class="toc-item-num">2.2.3&nbsp;&nbsp;</span>Bad Rating Dictionary</a></span></li></ul></li><li><span><a href="#LDA-Run" data-toc-modified-id="LDA-Run-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>LDA Run</a></span><ul class="toc-item"><li><span><a href="#LDA-Bad-Ratings" data-toc-modified-id="LDA-Bad-Ratings-2.3.1"><span class="toc-item-num">2.3.1&nbsp;&nbsp;</span>LDA Bad Ratings</a></span></li></ul></li></ul></li></ul></div>


# Part 3.2: LDA Analysis On Comments Based On Overall Review
---

In [56]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
from sklearn.decomposition import LatentDirichletAllocation

from gensim import corpora, models
import pyLDAvis
from langdetect import detect

pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

np.random.seed(42)

## Import Review Data 
---

#### Import Review Data Steps 
1. import data
2. select 2 columns: reivews, and sentiment 
3. break dataframe into 2, one with positive sentiment, the other with negative 
4. run LDA on both 

In [57]:
review_sentiments = pd.read_csv("../data/reviews_sentiment_score.csv", index_col = 0)

In [58]:
review_sentiments.head()

,key_0,listing_id,id,date,reviewer_id,reviewer_name,comments,language,overall_rating,compound,neg,neu,pos
0,0,958,5977,2009-07-23,15695,Edmund C,"Our experience was, without a doubt, a five st...",en,97.0,0.9590,0.000,0.788,0.212
1,1,958,6660,2009-08-03,26145,Simon,Returning to San Francisco is a rejuvenating t...,en,97.0,0.9819,0.000,0.697,0.303
2,2,958,11519,2009-09-27,25839,Denis,We were very pleased with the accommodations a...,en,97.0,0.7600,0.134,0.710,0.156
3,3,958,16282,2009-11-05,33750,Anna,We highly recommend this accomodation and agre...,en,97.0,0.9840,0.035,0.646,0.319
4,4,958,26008,2010-02-13,15416,Venetia,Holly's place was great. It was exactly what I...,en,97.0,0.9617,0.000,0.613,0.387


In [59]:
review_sentiments.dtypes

key_0               int64
listing_id          int64
id                  int64
date               object
reviewer_id         int64
reviewer_name      object
comments           object
language           object
overall_rating    float64
compound          float64
neg               float64
neu               float64
pos               float64
dtype: object

In [60]:
good_rating = review_sentiments[review_sentiments['overall_rating'] == 100 ]

In [61]:
good_rating.head()

,key_0,listing_id,id,date,reviewer_id,reviewer_name,comments,language,overall_rating,compound,neg,neu,pos
4598,4791,25094,81140853,2016-06-21,76212897,Mattie,Bruce and Alfredo were wonderful hosts. I felt...,en,100.0,0.9853,0.033,0.695,0.272
4599,4792,25094,86021875,2016-07-14,6770350,Patty,I have stayed in Airbnb's all over the US and ...,en,100.0,0.9792,0.027,0.621,0.352
4600,4793,25094,88188179,2016-07-23,83842908,Brian,Bruce and Alfredo were the best hosts I've had...,en,100.0,0.9826,0.019,0.742,0.240
4601,4794,25094,101684726,2016-09-14,6937471,Keely,Bruce and Alfredo are the most wonderful hosts...,en,100.0,0.9565,0.000,0.715,0.285
4602,4795,25094,103330395,2016-09-21,36866721,Hyejin,Bruce and Alfredo are amazing hosts. They welc...,en,100.0,0.9878,0.000,0.463,0.537


In [62]:
good_rating.shape

(13949, 13)

In [63]:
bad_rating = review_sentiments[review_sentiments['overall_rating'] < 80]

In [64]:
bad_rating.shape

(606, 13)

## LDA Prep
---

### Step1: CountVectorize 

#### Good Rating Dictionary 
---

In [65]:
#for positive sentiment 
cv = CountVectorizer(ngram_range= (2,2), 
                     stop_words= 'english', 
                     min_df = 2)
cv_good_rating = cv.fit_transform(good_rating['comments'])

In [66]:
cv_goodrating_df =  pd.DataFrame(cv_good_rating.toarray(), columns = cv.get_feature_names())

In [67]:
good_texts = [cv_goodrating_df.columns[cv_goodrating_df.loc[index,:].to_numpy().nonzero()] for index in cv_goodrating_df.index] 

In [68]:
dictionary_good = corpora.Dictionary(good_texts)

In [69]:
dictionary_good

In [70]:
corpus_good = [dictionary_good.doc2bow(text) for text in good_texts]

### LDA On Good Rating 

In [71]:
ldamodel_good = models.ldamodel.LdaModel(corpus_good,                     # pass in our corpus
                                    id2word = dictionary_good,       # matches each word to its "number" or "spot" in the dictionary
                                    num_topics = 15,             # number of topics T to find
                                    passes = 5,                 # number of passes through corpus; similar to number of epochs
                                    minimum_probability = 0.01)

In [72]:
pyLDAvis.gensim.prepare(ldamodel_good, corpus_good, dictionary_good)

/Users/evelyn/anaconda3/lib/python3.7/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.135333 -0.087448       1        1  11.171629
9     -0.122987  0.061476       2        1   9.910670
0     -0.102934 -0.154166       3        1   9.452935
4     -0.119918  0.087751       4        1   9.270441
5     -0.052866 -0.014179       5        1   7.315911
12    -0.011146 -0.063444       6        1   7.179464
11    -0.055087 -0.088678       7        1   6.916806
8     -0.051797  0.243002       8        1   6.842409
7     -0.028651  0.023613       9        1   6.357165
3     -0.004999  0.012742      10        1   6.336643
1      0.082894  0.012551      11        1   4.660483
14     0.129878 -0.024115      12        1   4.261859
10     0.114628 -0.013401      13        1   4.258588
6      0.175025 -0.024865      14        1   3.463933
13     0.183294  0.029161      15        1   2.601064, topic_info=      Category         Freq                  Term        Total  loglift  \
314    Default  1537.000000        great location  1537.000000  30.0000   
1941   Default   574.000000           golden gate   574.000000  29.0000   
402    Default   864.000000           great place   864.000000  28.0000   
1940   Default   448.000000             gate park   448.000000  27.0000   
703    Default   646.000000           place great   646.000000  26.0000   
383    Default   996.000000      walking distance   996.000000  25.0000   
51     Default  1125.000000            place stay  1125.000000  24.0000   
353    Default  1835.000000      highly recommend  1835.000000  23.0000   
277    Default   213.000000             away home   213.000000  22.0000   
386    Default   385.000000           best airbnb   385.000000  21.0000   
734    Default  1044.000000            great host  1044.000000  20.0000   
105    Default   264.000000             ve stayed   264.000000  19.0000   
287    Default   193.000000             home away   193.000000  18.0000   
291    Default   346.000000             like home   346.000000  17.0000   
961    Default   185.000000          days arrival   185.000000  16.0000   
3096   Default   313.000000         quick respond   313.000000  15.0000   
155    Default   182.000000     automated posting   182.000000  14.0000   
154    Default   182.000000     arrival automated   182.000000  13.0000   
156    Default   177.000000  canceled reservation   177.000000  12.0000   
158    Default   177.000000         host canceled   177.000000  11.0000   
57     Default  2775.000000         san francisco  2775.000000  10.0000   
147    Default   501.000000          enjoyed stay   501.000000   9.0000   
664    Default   973.000000       definitely stay   973.000000   8.0000   
1716   Default   739.000000  definitely recommend   739.000000   7.0000   
3287   Default   397.000000        really enjoyed   397.000000   6.0000   
283    Default   309.000000             feel like   309.000000   5.0000   
1687   Default   120.000000         friendly host   120.000000   4.0000   
3087   Default   255.000000            great view   255.000000   3.0000   
180    Default   153.000000            right home   153.000000   2.0000   
1374   Default   424.000000           super clean   424.000000   1.0000   
...        ...          ...                   ...          ...      ...   
3398   Topic15    20.053862      great definitely    21.212692   3.5931   
13964  Topic15    19.625832         lovely little    20.784668   3.5919   
1732   Topic15    19.373684          park walking    20.536266   3.5910   
14911  Topic15    18.193516        shown pictures    19.352730   3.5875   
931    Topic15    16.979824           lots little    18.139465   3.5832   
4064   Topic15    18.749903       access downtown    20.033255   3.5830   
10018  Topic15    16.756975    shower comfortable    17.915800   3.5824   
17442  Topic15    16.128630            park right    17.287449   3.5799   
9385   Topic15    15.950531            

### Bad Rating Dictionary 
---

In [73]:
#for positive sentiment 
cv = CountVectorizer(ngram_range= (2,2), 
                     stop_words= 'english', 
                     min_df = 2)
cv_bad_rating = cv.fit_transform(bad_rating['comments'])

In [75]:
cv_bad_rating_df =  pd.DataFrame(cv_bad_rating.toarray(), columns = cv.get_feature_names())

In [76]:
bad_rating_texts = [cv_bad_rating_df.columns[cv_bad_rating_df.loc[index,:].to_numpy().nonzero()] for index in cv_bad_rating_df.index]

In [77]:
dictionary_bad = corpora.Dictionary(bad_rating_texts)

In [78]:
corpus_bad = [dictionary_bad.doc2bow(text) for text in bad_rating_texts]

## LDA Run
---

### LDA Bad Ratings

In [79]:
ldamodel_bad = models.ldamodel.LdaModel(corpus_bad, 
                                        id2word = dictionary_bad, 
                                        num_topics = 15, 
                                        passes = 5, 
                                        minimum_probability = 0.01)

In [80]:
pyLDAvis.gensim.prepare(ldamodel_bad, corpus_bad, dictionary_bad)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14    -0.106009 -0.066672       1        1  10.558556
0      0.263217 -0.060654       2        1   9.948191
13    -0.030843 -0.093459       3        1   8.905152
4     -0.011323  0.054965       4        1   8.005049
3      0.022284  0.068084       5        1   6.579798
12    -0.077586 -0.045724       6        1   6.279402
2      0.012415 -0.055036       7        1   6.163864
10     0.019800  0.087382       8        1   6.051888
11    -0.046255 -0.030564       9        1   5.973757
1     -0.055734  0.055212      10        1   5.752614
8     -0.022278  0.172760      11        1   5.680187
7      0.015097  0.000026      12        1   5.608445
5     -0.025337 -0.059775      13        1   5.588699
9      0.059883  0.052616      14        1   4.639785
6     -0.017332 -0.079162      15        1   4.264619, topic_info=     Category       Freq                   Term      Total  loglift  logprob
466   Default  20.000000      automated posting  20.000000  30.0000  30.0000
465   Default  20.000000      arrival automated  20.000000  29.0000  29.0000
468   Default  19.000000           days arrival  19.000000  28.0000  28.0000
469   Default  16.000000          host canceled  16.000000  27.0000  27.0000
467   Default  16.000000   canceled reservation  16.000000  26.0000  26.0000
78    Default  31.000000         great location  31.000000  25.0000  25.0000
294   Default  16.000000            great value  16.000000  24.0000  24.0000
11    Default  27.000000             place stay  27.000000  23.0000  23.0000
842   Default  48.000000        homeless people  48.000000  22.0000  22.0000
42    Default  17.000000       walking distance  17.000000  21.0000  21.0000
295   Default  10.000000            place clean  10.000000  20.0000  20.0000
470   Default   7.000000       reservation days   7.000000  19.0000  19.0000
182   Default  11.000000          good location  11.000000  18.0000  18.0000
68    Default  19.000000             room clean  19.000000  17.0000  17.0000
4     Default  14.000000             great host  14.000000  16.0000  16.0000
150   Default  13.000000            great place  13.000000  15.0000  15.0000
802   Default   8.000000       central location   8.000000  14.0000  14.0000
417   Default  15.000000           union square  15.000000  13.0000  13.0000
219   Default   9.000000             good place   9.000000  12.0000  12.0000
67    Default  15.000000         location great  15.000000  11.0000  11.0000
367   Default   9.000000             good value   9.000000  10.0000  10.0000
94    Default   9.000000            minute walk   9.000000   9.0000   9.0000
13    Default  40.000000          san francisco  40.000000   8.0000   8.0000
10    Default   8.000000             nice place   8.000000   7.0000   7.0000
698   Default   4.000000              make sure   4.000000   6.0000   6.0000
921   Default   7.000000          people street   7.000000   5.0000   5.0000
2     Default   8.000000              gate park   8.000000   4.0000   4.0000
45    Default   8.000000            blocks away   8.000000   3.0000   3.0000
70    Default   9.000000       shared bathrooms   9.000000   2.0000   2.0000
736   Default   6.000000             place good   6.000000   1.0000   1.0000
...       ...        ...                    ...        ...      ...      ...
974   Topic15   0.737345            stay really   2.189566   2.0664  -5.3388
815   Topic15   0.737351             room super   2.195812   2.0636  -5.3388
493   Topic15   0.737349          stayed nights   2.195830   2.0636  -5.3388
162   Topic15   0.737352         location block   2.195839   2.0636  -5.3388
441   Topic15   0.737350       stay comfortable   2.195843   2.0636  -5.3388
1005  Topic15   0.737306        room advertised   2.195788   2.0635  -5.3389
135   Topic15   0.737352            stay highly   2.212412   2.0560  -5.3388
44    Topic15   0.737357            